In [1]:
import shutil
import sys
import os
from pathlib import Path
import json
import re

root_path = Path('/home/jianghui/dataset/MovieNet-Subset/')
shot_root_path = Path(root_path, 'all/shot/')
scene_root_path = Path(root_path, 'all/scene/')
subtitle_root_path = Path(root_path, 'all/subtitle/')
synopsis_root_path = Path(root_path, 'all/synopsis/')
cast_root_path = Path(root_path, 'all/cast/')
names = locals()

tt_list = []
with open(Path(root_path, 'movie_id_list'), 'r') as f_tt_list:
    for tt_line in f_tt_list:
        tt_list.append(tt_line.replace('\n', ''))

def open_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'r') as names[f'f_{types}']:
        names[f'{types}_json'] = json.load(names[f'f_{types}'])
    return names[f'{types}_json']

def save_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'w') as names[f'f_{types}']:
        json.dump(names[f'{types}_json'], names[f'f_{types}'], indent=2)

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    shot_json = open_json('shot', tt_id)
    scene_json = open_json('scene', tt_id)

    for scene_ele in scene_json:
        for shot_id in range(scene_ele['shot_id'][0], scene_ele['shot_id'][1] + 1):
            shot_json[shot_id]['scene_id'] = scene_ele['scene_id']

    save_json('shot', tt_id)

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    shot_json = open_json('shot', tt_id)
    synopsis_json = open_json('synopsis', tt_id)

    for synopsis_ele in synopsis_json:
        scene_start_id = shot_json[synopsis_ele['shot_id'][0]]['scene_id']
        scene_end_id = shot_json[synopsis_ele['shot_id'][1]]['scene_id']
        synopsis_ele['scene_id']=[scene_start_id, scene_end_id]
        
    save_json('synopsis', tt_id)

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    synopsis_json = open_json('synopsis', tt_id)
    
    for synopsis_ele in synopsis_json:
        synopsis_ele['sentence'] = synopsis_ele.pop('scentence')

    save_json('synopsis', tt_id)

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    subtitle_json = open_json('subtitle', tt_id)
    
    for subtitle_ele in subtitle_json:
        subtitle_ele['sentence'] = subtitle_ele.pop('scentence')

    save_json('subtitle', tt_id)

In [2]:
import re


In [3]:
prog_shot = re.compile('(?<=shot_)\d+(?=_img_)')
prog_img = re.compile('(?<=img_)\d+(?=.jpg)')
string = 'shot_0003_img_0.jpg'
prog_shot.search(string).group(0), prog_img.search(string).group(0)

('0003', '0')

In [4]:
movienet_path = Path('/home/jianghui/dataset/MovieNet/')
movienet_img_path = Path(movienet_path, "movie1K.keyframes.240p.v1/240P/")



In [ ]:
for tt_id in tt_list:
    src_dir = Path(movienet_img_path, tt_id)
    dst_dir = Path(root_path, 'all/img/', tt_id)

    Path.mkdir(dst_dir, exist_ok=True)

    for src_file in src_dir.iterdir():
        shot_id_match = prog_shot.search(src_file.name)
        img_id_match = prog_img.search(src_file.name)

        if shot_id_match != None:
            shot_id = int(shot_id_match.group(0))
            img_id = int(img_id_match.group(0))

            dst_file = Path(dst_dir, f'{shot_id}_{img_id}.jpg')

            shutil.copyfile(src_file, dst_file)